Prepare Dataset

In [1]:
!git clone https://github.com/danielphamvt/sentiment_analysis_nal.git

Cloning into 'sentiment_analysis_nal'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Receiving objects: 100% (33/33), 1.19 MiB | 7.78 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
!mkdir dataSA

mkdir: cannot create directory ‘dataSA’: File exists


In [3]:
!mv /content/sentiment_analysis_nal/data_clean/train.crash /content/dataSA

In [4]:
!mv /content/sentiment_analysis_nal/data_clean/test.crash /content/dataSA

In [5]:
!rm -r sentiment_analysis_nal

Prepare transformer and library

In [6]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [7]:
# !cd transformers


In [8]:
!pip install fastBPE

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
# cant install fairseq with pip 24.2
!pip install pip==24.0

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
!pip install fairseq

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


fastBPE is a tool for efficient tokenization using the BPE algorithm, while fairseq is a versatile toolkit for training and deploying state-of-the-art NLP models

In [11]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [12]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
--2024-07-31 16:35:17--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   147MB/s    in 0.2s    

2024-07-31 16:35:17 (147 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2024-

In [13]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

text = "Tôi là học sinh của trường Khoa học Tự Nhiên TPHCM. Tôi thích mèo."

word_segmented_text = rdrsegmenter.tokenize(text)
print(word_segmented_text)


[['Tôi', 'là', 'học_sinh', 'của', 'trường', 'Khoa_học_Tự_Nhiên', 'TPHCM', '.'], ['Tôi', 'thích', 'mèo', '.']]


rdrsegmenter is basically the word tokenizer for vnmese

Note that the tokenizer can split between sentence

In [14]:
# installing the pretrain for phoBERT
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz


--2024-07-31 16:35:24--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.160.41.47, 18.160.41.104, 18.160.41.77, ...
Connecting to public.vinai.io (public.vinai.io)|18.160.41.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz.1’

PhoBERT_base_fairse 100%[===================>]   1.16G  85.1MB/s    in 13s     

2024-07-31 16:35:37 (90.9 MB/s) - ‘PhoBERT_base_fairseq.tar.gz.1’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [15]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz



--2024-07-31 16:36:06--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.230.31.108, 54.230.31.74, 54.230.31.76, ...
Connecting to public.vinai.io (public.vinai.io)|54.230.31.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz.1’

PhoBERT_base_transf 100%[===================>] 307.47M  37.7MB/s    in 4.7s    

2024-07-31 16:36:11 (65.4 MB/s) - ‘PhoBERT_base_transformers.tar.gz.1’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [16]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

In [17]:
## following the instructions of phoBERT
parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
    default="/content/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/PhoBERT_base_transformers/dict.txt")

In [18]:
bpe.encode('Tôi là học sinh của trường Khoa học Tự Nhiên TPHCM. Tôi thích mèo.')

'Tôi là học sinh của trường Khoa học Tự Nhiên TP@@ HCM. Tôi thích m@@ è@@ o.'

In [19]:
vocab.encode_line('<s> ' + 'Tôi là học sinh của trường Khoa học Tự Nhiên TP@@ HCM. Tôi thích m@@ è@@ o.' + ' </s>')


tensor([    0,   218,     8,   222,   418,     7,   212,  3720,   222,  4544,
        12567,  8656, 54251,   218,   543,  1387,  9367, 35211,     2,     2],
       dtype=torch.int32)

`<s>` là token đặc biệt để đánh dấu vị trí đầu câu và `</s>` để đánh dấu vị trí cuối mỗi câu.

In [20]:
import re
train_path = "/content/dataSA/train.crash"
test_path = "/content/dataSA/test.crash"

In [21]:
train_id, train_text, train_label = [], [], []
test_id, test_text = [], []

train_000000  
"Dung dc sp tot cam on  
shop Đóng gói sản phẩm rất đẹp và chắc chắn Chất lượng sản phẩm tuyệt vời"                
0

In [22]:
with open(train_path, 'r') as f_r:
    data = f_r.read().strip()
    data = re.findall('train_[\s\S]+?\"\n[01]\n\n', data)
    # get the text between " "
    for sample in data:
        splits = sample.strip().split('\n')

        id = splits[0] #train_000000
        label = int(splits[-1]) #0
        text = ' '.join(splits[1:-1])[1:-1] # join each sentence with sapce and drop " "
        text = rdrsegmenter.tokenize(text) # tokenize
        text = ' '.join([' '.join(x) for x in text]) # join with space

        train_id.append(id)
        train_text.append(text)
        train_label.append(label)

In [23]:
with open(test_path, 'r') as f_r:
    data = f_r.read().strip()
    data = re.findall('test_[\s\S]+?\"\n\n', data)
    for sample in data:
        splits = sample.strip().split('\n')

        id = splits[0]
        text = ' '.join(splits[1:])[1:-1]
        text = rdrsegmenter.tokenize(text)
        text = ' '.join([' '.join(x) for x in text])

        test_id.append(id)
        test_text.append(text)

In [24]:
import numpy as np
print(len(train_id),len(test_id))
print(np.unique(train_label, return_counts=True))

16071 10980
(array([0, 1]), array([8689, 7382]))


In [25]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_label, test_size=0.1)


In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 125

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)

train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")


use bpe to encode the vnmese to phrase words and use vocab to map it to its id

The pad_sequences will truncated the redundant and padding to ensure that the vector will be 125 long

In [28]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)


The masks (used for transformer) includes 0 and 1, tell the model which token is padding

In [37]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)


SequentialSampler is used to ensure that the data is loaded in a sequential order. This is often used during validation/testing when the order of samples matters or when you want deterministic results.

In [38]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/content/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 2, output_hidden_states=False,
)
BERT_SA = BertForSequenceClassification.from_pretrained(
    "/content/PhoBERT_base_transformers/model.bin",
    config=config
)
BERT_SA.cuda()


You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


NameError: name 'BertForSequenceClassification' is not defined